In [ ]:
!pip install nltk gensim pennylane pennylane-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 898.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing ins

In [ ]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import pennylane as qml
from pennylane import numpy as np
import nltk
from pennylane.optimize import AdamOptimizer
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#@title Word Embedding
sentences = ["Dynex powers Quantum entanglement", "Neuromorphic networks process qubits"]
tokenization = [word_tokenize(sentence.lower()) for sentence in sentences]
word2vec = Word2Vec(sentences=tokenization, vector_size=8, window=5, min_count=1, workers=4)

In [ ]:
#@title Basis Embedding
qubits = 8
dev = qml.device("default.qubit", wires=qubits)

In [ ]:
#@title Quantum Self-Attention Layers

@qml.qnode(dev)
def QuantumSelfAttention(inputs, weights=np.random.randn(3, qubits, 3)):
    # biIn = [1 if x >= 0 else 0 for x in inputs]
    # qml.BasisEmbedding(biIn, wires=range(qubits))
    qml.AngleEmbedding(inputs, wires=range(qubits))

    """
    Rotation Gates: RX, RY, and RZ gates are applied to each qubit. These gates rotate the qubits around the X, Y, and Z axes respectively. The angles of rotation are determined by the values in the input vector. This step effectively processes each feature of the word embeddings, manipulating the quantum state to capture relationships between different dimensions.
    Entanglement: CRZ and CNOT gates create entanglement between qubits, allowing the circuit to consider interactions between different features. Entanglement is a uniquely quantum phenomenon that links the states of qubits, so the state of one qubit depends on the state of another.
    """

    for layer in range(3):
        for i in range(qubits):
            qml.RX(weights[layer][i][0] * inputs[i % len(inputs)], wires=i)
            qml.RY(weights[layer][i][1] * inputs[(i + 1) % len(inputs)], wires=i)
            qml.RZ(weights[layer][i][2] * inputs[(i + 2) % len(inputs)], wires=i)
        for i in range(qubits - 1):
            qml.CRZ(np.pi / (layer + 2), wires=[i, (i + 1) % qubits])
            qml.CNOT(wires=[i, (i + 1) % qubits])

    """
    Grover’s Operator
    """
    qml.QFT(wires=range(qubits))
    qml.adjoint(qml.QFT)(wires=range(qubits))
    qml.GroverOperator(wires=range(qubits))

    """
    Final Quantum Operations
    """

    for i in range(qubits):
        qml.Hadamard(wires=i)
        qml.T(wires=i)
        qml.RZ(inputs[i % len(inputs)], wires=i)

        # qml.BasisEmbedding(biIn, wires=range(qubits))
        qml.AngleEmbedding(inputs, wires=range(qubits))
        return [qml.expval(qml.PauliZ(wires=i)) for i in range(qubits)]

In [ ]:
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def loss_fn(weights, input_vector, target_vector):
    output = QuantumSelfAttention(input_vector, weights)

    # Adjust dimension mismatch
    if len(output) != len(target_vector):
        adjusted = np.zeros_like(target_vector)
        adjusted[:len(output)] = output
    else:
        adjusted = output

    # Cosine distance (semakin kecil semakin bagus)
    cos_sim = np.dot(adjusted, target_vector) / (np.linalg.norm(adjusted) * np.linalg.norm(target_vector))
    return 1 - cos_sim


def find_most_similar_word(vector):
    """Menemukan kata yang paling mirip dengan vektor yang diberikan"""
    most_similar = None
    max_similarity = -float('inf')

    # Iterasi melalui semua kata dalam model Word2Vec
    for word in word2vec.wv.index_to_key:
        similarity = np.dot(word2vec.wv[word], vector) / (
            np.linalg.norm(word2vec.wv[word]) * np.linalg.norm(vector))

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar = word

    return most_similar

def GenerateSentence(input_):
    tokens = word_tokenize(input_.lower())
    iEmbeddings = np.array([word2vec.wv[word] for word in tokens])
    attOutputs = []

    for embedding in iEmbeddings:
        attOUT = QuantumSelfAttention(embedding)
        softOUT = softmax(attOUT)
        attOutputs.append(softOUT)

    # Mengubah attOutputs menjadi kata-kata
    generated_words = []
    for vector in attOutputs:
        # Gunakan attOUT sebagai vektor baru untuk mencari kata terdekat
        # Karena attOUT adalah vektor hasil kuantum dengan dimensi 8 (jumlah qubit)
        # kita perlu memastikan dimensinya cocok dengan word2vec
        if len(vector) != word2vec.vector_size:
            # Jika dimensi berbeda, potong atau tambah nilai untuk menyesuaikan
            adjusted_vector = np.zeros(word2vec.vector_size)
            for i in range(min(len(vector), word2vec.vector_size)):
                adjusted_vector[i] = vector[i]
        else:
            adjusted_vector = vector

        # Cari kata yang paling mirip dengan vektor yang dihasilkan
        similar_word = find_most_similar_word(adjusted_vector)
        generated_words.append(similar_word)

    # Tambahkan beberapa kata dari kosakata untuk variasi jika diperlukan
    if len(generated_words) < 4 and len(word2vec.wv.index_to_key) > 0:
        additional_needed = 4 - len(generated_words)
        # Ambil beberapa kata acak dari kosakata
        import random
        vocab_list = list(word2vec.wv.index_to_key)
        for _ in range(min(additional_needed, len(vocab_list))):
            generated_words.append(random.choice(vocab_list))

    return ' '.join(generated_words[:5])

# tes generated

In [ ]:
# input_ = np.random.choice(sentences)
input_ = "qubits"
generated = GenerateSentence(input_)
print(f"Generated: {generated}")

Generated: qubits networks neuromorphic entanglement


# training

In [ ]:
from gensim.test.utils import common_texts
word2vec = Word2Vec(sentences=common_texts, vector_size=8, window=5, min_count=1, workers=1)


In [ ]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [ ]:
import pandas as pd

data = pd.DataFrame({
    "input": ["human", "computer", "system", "trees"],
    "target": ["user", "system", "computer", "graph"]
})


In [ ]:
# === Preprocess training data ===
training_data = []
missing_words = []

for _, row in data.iterrows():
    try:
        input_vec = word2vec.wv[row["input"]]
        target_vec = word2vec.wv[row["target"]]
        training_data.append((input_vec, target_vec))
    except KeyError as e:
        missing_words.append(str(e))

In [ ]:
from pennylane.optimize import NesterovMomentumOptimizer

num_layers = 3
epochs = 10
opt = AdamOptimizer(stepsize=0.01)
weights = np.random.randn(num_layers, qubits, 3)  # initial weights
opt = NesterovMomentumOptimizer(stepsize=0.01)

if len(training_data) == 0:
    print("❌ Tidak ada pasangan kata yang valid di Word2Vec! Berikut kata yang tidak ditemukan:")
    print(missing_words)
else:
    print(f"✅ Total pasangan data untuk training: {len(training_data)}")

    # === Training ===
    for epoch in range(epochs):
        loss_total = 0
        for input_vec, target_vec in training_data:
            weights, loss = opt.step_and_cost(lambda w: loss_fn(w, input_vec, target_vec), weights)
            loss_total += loss
        print(f"Epoch {epoch + 1} - Loss: {loss_total / len(training_data):.4f}")

✅ Total pasangan data untuk training: 4


/usr/local/lib/python3.11/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Epoch 1 - Loss: 1.1197
Epoch 2 - Loss: 1.1197
Epoch 3 - Loss: 1.1197
Epoch 4 - Loss: 1.1197
Epoch 5 - Loss: 1.1197
Epoch 6 - Loss: 1.1197
Epoch 7 - Loss: 1.1197
Epoch 8 - Loss: 1.1197
Epoch 9 - Loss: 1.1197
Epoch 10 - Loss: 1.1197
